# Exercises Editor
Author: Brent Artuch <br>
Date: 2023-06-07

## Import From SQL

## Introduction

In this notebook we explore how exercise data is logged for a given day of training and calculations to
determine the output of each exercise and what the desired output for the following week will be. The
data entries are recorded at the gym using a simple log app that I developed for Android. The key data
in this notebook will be the `weight` and `reps` of each exercise.

## Step: 1 Imports
### Import packages

In [82]:
import sqlite3
import pandas as pd
import plotly.express as px

### Load the dataset
This dataset is pulled from a SQLite database named `exercises` with four tables: `leg_day`, `chest_day`,
`back_day`, and `shoulder_day`. We will load the desired day using a simple query.

In [83]:
sql_connection = sqlite3.connect('exercises.sqlite')
# Import desired table and set the index.
current_exercises = pd.read_sql('SELECT * FROM back_day', sql_connection, index_col='index')
current_exercises.tail(10)

,id,date,name,weight,reps
index,,,,,
35,1000036,2023-05-29 00:00:00,Bicep 21,42,21
36,1000037,2023-06-05 00:00:00,Platform Back Raises,135,17
37,1000038,2023-06-05 00:00:00,Straight Arm Pulldown,80,12
38,1000039,2023-06-05 00:00:00,Lat Pulldown,260,7
39,1000040,2023-06-05 00:00:00,Smith Rows,290,7
40,1000041,2023-06-05 00:00:00,High Row Machine,150,8
41,1000042,2023-06-05 00:00:00,Rear Delt Machine,190,10
42,1000043,2023-06-05 00:00:00,Preacher Curl Machine,140,15
43,1000044,2023-06-05 00:00:00,Single Arm Curl,25,11


## Step 2: Data Exploration
Next we will get a sense of the size and contents of `current_exercises`.

In [84]:
current_exercises.shape

(45, 5)

In [85]:
current_exercises.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45 non-null     int64 
 1   date    45 non-null     object
 2   name    45 non-null     object
 3   weight  45 non-null     object
 4   reps    45 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [86]:
current_exercises['name'].value_counts()

Platform Back Raises     5
Straight Arm Pulldown    5
Lat Pulldown             5
Smith Rows               5
High Row Machine         5
Rear Delt Machine        5
Preacher Curl Machine    5
Single Arm Curl          5
Bicep 21                 5
Name: name, dtype: int64

## Step 3: Update the Dataset
First we will get each of the unique names from `current_exercises`.

In [87]:
# Set names to a list of unique exercise names
names = current_exercises['name'].unique()
names

array(['Platform Back Raises', 'Straight Arm Pulldown', 'Lat Pulldown',
       'Smith Rows', 'High Row Machine', 'Rear Delt Machine',
       'Preacher Curl Machine', 'Single Arm Curl', 'Bicep 21'],
      dtype=object)

Next we set the date that we wish to update.

In [88]:
day_of_month = input("Enter the day of the month: DD")
date = f"2023-06-{day_of_month} 00:00:00"
date

'2023-06-12 00:00:00'

### Execute the entry insertion algorithm

In [89]:
for i in range(0, len(names)):

  """ This algorithm sets the elements of a new entry with user input and then inserts it into the
  database table for each name. Id is calculated by last index."""

  weight = input(f"Enter Weight for {names[i]}: ")
  reps = input(f"Enter Reps for {names[i]}: ")
  e_id = current_exercises.iloc[-1, 0] + 1
  new_entry = [e_id, date, names[i], weight, reps]
  current_exercises.loc[len(current_exercises)] = new_entry

# Format the string dates to datetime.
current_exercises['date'] = pd.to_datetime(current_exercises['date'])
current_exercises.tail(10)

,id,date,name,weight,reps
index,,,,,
44,1000045,2023-06-05,Bicep 21,50,21
45,1000046,2023-06-12,Platform Back Raises,135,17
46,1000047,2023-06-12,Straight Arm Pulldown,87,12
47,1000048,2023-06-12,Lat Pulldown,220,12
48,1000049,2023-06-12,Smith Rows,290,10
49,1000050,2023-06-12,High Row Machine,120,12
50,1000051,2023-06-12,Rear Delt Machine,205,9
51,1000052,2023-06-12,Preacher Curl Machine,140,15
52,1000053,2023-06-12,Single Arm Curl,30,10


### Data Manipulation
This section exists for convenience if quick changes need to be made to the entries.

In [ ]:
# Update a single item by id and column
# current_exercises.loc[39, 'weight'] = 290
# Drop last row
# current_exercises.drop(index=current_exercises.index[-1], axis=0, inplace=True)
current_exercises.tail(10)

### Update SQL
Now that we have updated our table, we will export the new dataset to SQL. As a precaution, I always
back up the new dataset to CSV through the SQL workbench in the event that an older version is required
in the future.

In [90]:
# Export new chart to database with a date stamp
current_exercises.to_sql('back_day', sql_connection, if_exists='replace')

# TODO: MANUALLY REVIEW AND EXPORT NEW TABLE TO CSV AS BACKUP!!

54

## Step 5: Calculations
Since, I rotate my weights and reps each weak based on performance, we use the volume equation
to determine the rep to weight output.<br>
$Volume$ = `weight` $\cdot$ `reps` $\cdot$ $sets$ <br>
For our purposes the sets parameter will always be 1 because only one working set is performed.

In [91]:
current_exercises['volume'] = current_exercises['weight'].astype(int) * current_exercises['reps'].astype(int)
current_exercises.head()

,id,date,name,weight,reps,volume
index,,,,,,
0,1000001,2023-04-17,Platform Back Raises,135,13,1755
1,1000002,2023-04-17,Straight Arm Pulldown,95,9,855
2,1000003,2023-04-17,Lat Pulldown,260,8,2080
3,1000004,2023-04-17,Smith Rows,290,6,1740
4,1000005,2023-04-17,High Row Machine,130,11,1430


##### Divide the exercises into two groups `low_volume` and `high_volume`.

## Step 6: Visualization
Next, plot the volume trend for the exercise.

In [98]:
fig = px.line(
  current_exercises, x='date', y='volume', color='name', markers=True,
  height=600,width=800,
  labels={
    'volume': "Weight x Reps",
    'date': "Workout Date",
    'name': "Exercise"
  },
  title="Back Day Exercises"
)
fig.update_traces(textposition='top center')
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

## Leg Day Notes:
* 2023-05-19: Changed Smith Squat range of motion to full so should be marked drop in weight.
* 2023-06-02: Changed Leg Extensions from two legs to one which will account for the weight change.